In [1]:
import pandas as pd

In [2]:
!ls data/

gender_submission.csv  test.csv  train.csv


In [3]:
train = pd.read_csv('data/train.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test = pd.read_csv('data/test.csv')

In [6]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
train.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [8]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [9]:
train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [10]:
test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,0,1


In [11]:
train['Age'].fillna(train['Age'].mean(), inplace=True)
test['Age'].fillna(test['Age'].mean(), inplace=True)

In [12]:
test['Fare'].fillna(test['Fare'].mean(), inplace=True)

In [13]:
X = train.drop('Survived', axis=1)
y = train['Survived']

In [14]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20)

In [60]:
fit_params={
    "early_stopping_rounds":30, 
    "eval_metric" : 'auc', 
    "eval_set" : [(X_val, y_val)],
    'eval_names': ['valid'],
    'verbose': 100,
    'callbacks': [lgb.reset_parameter(learning_rate=lambda i: np.power(.995, i))],
    'categorical_feature': 'auto'
}

In [61]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [62]:
param_val ={
    'num_leaves': sp_randint(6, 50), 
    'min_child_samples': sp_randint(100, 500), 
    'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
    'subsample': sp_uniform(loc=0.2, scale=0.8), 
    'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
    'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
    'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]
}

In [70]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [71]:
clf = lgb.LGBMClassifier(max_depth=-1, silent=True, metric='None',
                         n_jobs=4, n_estimators=5000, min_data=1, min_data_in_bin=1)

In [76]:
rs = RandomizedSearchCV(
    estimator=clf, 
    param_distributions=param_val, 
    n_iter=100,
    scoring='roc_auc', 
    cv=5,
    refit=True,
)

In [77]:
X_train.shape

(712, 11)

In [78]:
X_val.shape

(179, 11)

In [79]:
rs.fit(X_train, y_train, **fit_params)
print('Best score: {} - Params: {} '.format(rs.best_score_, rs.best_params_))

Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.787247
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[2]	valid's auc: 0.781916
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[3]	valid's auc: 0.794376
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[3]	valid's auc: 0.802106
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[3]	valid's auc: 0.784515
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[6]	valid's auc: 0.808302
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[3]	valid's auc: 0.793843
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[8]	valid's auc: 0.803105
Training until v

Early stopping, best iteration is:
[21]	valid's auc: 0.823094
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[3]	valid's auc: 0.803971
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[5]	valid's auc: 0.817364
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[2]	valid's auc: 0.799174
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[2]	valid's auc: 0.782716
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[2]	valid's auc: 0.806903
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iter

Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[10]	valid's auc: 0.804171
Training until validation scores don't improve for

Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.736074
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.736074
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.736074
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.736074
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[

Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.736074
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.736074
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.736074
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.736074
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.736074
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.736074
Training until validation 

Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[3]	valid's auc: 0.81903
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[2]	valid's auc: 0.808635
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[3]	valid's auc: 0.815032
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[6]	valid's auc: 0.781183
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[3]	valid's auc: 0.804038
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores

Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 ro

Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 ro

In [80]:
opt_parameters = {
    'colsample_bytree': 0.9699607033936529, 
    'min_child_samples': 426, 
    'min_child_weight': 1, 
    'num_leaves': 11, 
    'reg_alpha': 1, 
    'reg_lambda': 20, 
    'subsample': 0.33517538499660093
}

In [81]:
clf.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, metric='None',
        min_child_samples=20, min_child_weight=0.001, min_data=1,
        min_data_in_bin=1, min_split_gain=0.0, n_estimators=5000, n_jobs=4,
        num_leaves=31, objective=None, random_state=None, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [82]:
submission = pd.DataFrame()
submission['PassengerId'] = test['PassengerId']
submission['Survived'] = clf.predict(test)

/home/pstwh/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [83]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [84]:
submission.to_csv('submission.csv', index=False)